In [2]:
from Model.CustomerModel import CustomerModel
from Model.SalesOrderModel import SalesOrderModel
from Connector.DatabaseConnection import DatabaseConnection

server = "localhost"
port = 3306
username="root"
database="northwind"
password="hoanganh123"

connector =DatabaseConnection(server=server, port=port, password=password, username=username, database=database)
sales_order_df = SalesOrderModel(connector).dataframe

{'employeeId': {'referenced_column': 'employeeId', 'model': <Model.EmployeeModel.EmployeeModel object at 0x00000157B613FEB0>}, 'shipperid': {'referenced_column': 'shipperId', 'model': <Model.ShipperModel.ShipperModel object at 0x00000157B613FF40>}, 'custId': {'referenced_column': 'custId', 'model': <Model.CustomerModel.CustomerModel object at 0x00000157B613FBE0>}}


In [3]:
sales_order_df.head()

,orderId,custId,employeeId,orderDate,requiredDate,shippedDate,shipperid,freight,shipName,shipAddress,shipCity,shipRegion,shipPostalCode,shipCountry
0,10248,85,5,04-Jul-2006,01-Aug-2006,16-Jul-2006,3,32.38,Ship to 85-B,6789 rue de l'Abbaye,Reims,None,10345,France
1,10249,79,6,05-Jul-2006,16-Aug-2006,10-Jul-2006,1,11.61,Ship to 79-C,Luisenstr. 9012,Münster,None,10328,Germany
2,10250,34,4,08-Jul-2006,05-Aug-2006,12-Jul-2006,2,65.83,Destination SCQXA,"Rua do Paço, 7890",Rio de Janeiro,RJ,10195,Brazil
3,10251,84,3,08-Jul-2006,05-Aug-2006,15-Jul-2006,1,41.34,Ship to 84-A,"3456, rue du Commerce",Lyon,None,10342,France
4,10252,76,4,09-Jul-2006,06-Aug-2006,11-Jul-2006,2,51.30,Ship to 76-B,"Boulevard Tirou, 9012",Charleroi,None,10318,Belgium


In [10]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

class KmeansService:
    def __init__(self, dataframe, num_k, is_rfm=None):
        """
        Initialize the KmeansService class with a dataframe, number of clusters (num_k), and an optional is_rfm flag.
        
        :param dataframe: pd.DataFrame - Data to be clustered
        :param num_k: int - Number of clusters
        :param is_rfm: bool - Optional flag for specific handling (e.g., RFM segmentation)
        """
        self.dataframe = dataframe
        self.num_k = num_k
        self.is_rfm = is_rfm
        self.model = None
        self.scaler = None

    def preprocess_data(self):
        """
        Preprocess the data by standardizing it.
        
        :return: np.ndarray - Scaled data
        """
        self.scaler = StandardScaler()
        scaled_data = self.scaler.fit_transform(self.dataframe)
        return scaled_data

    def train_model(self):
        """
        Train the KMeans model with the preprocessed data and add cluster labels to the dataframe.
        """
        # Preprocess the data
        data = self.preprocess_data()

        # Initialize and fit the KMeans model
        self.model = KMeans(n_clusters=self.num_k, random_state=42)
        self.model.fit(data)

        # Add cluster labels to the original dataframe
        self.dataframe['cluster'] = self.model.labels_

    def predict(self, new_data):
        """
        Predict the cluster for new data.
        
        :param new_data: pd.DataFrame - New data to be clustered
        :return: np.ndarray - Predicted cluster labels
        """
        # Preprocess the new data
        scaled_data = self.scaler.transform(new_data)

        # Predict the cluster for new data
        return self.model.predict(scaled_data)

    def get_centroids(self):
        """
        Get the centroids of the clusters.
        
        :return: np.ndarray - Centroids of the clusters
        """
        return self.model.cluster_centers_

    def get_clustered_data(self):
        """
        Get the original dataframe with an additional column for cluster labels.
        
        :return: pd.DataFrame - Dataframe with cluster labels
        """
        return self.dataframe

    def summarize_clusters(self):
        """
        Provide a summary of the clusters.
        
        :return: pd.DataFrame - Summary statistics of each cluster
        """
        return self.dataframe.groupby('cluster').mean()


   feature1  feature2  cluster
0         1         5        0
1         2         4        0
2         3         3        0
3         4         2        1
4         5         1        1
[0]
[[-0.70710678  0.70710678]
 [ 1.06066017 -1.06066017]]
         feature1  feature2
cluster                    
0             2.0       4.0
1             4.5       1.5
